In [ ]:
!pip install --upgrade accelerator

In [ ]:
!pip uninstall y transformers accelerate

In [ ]:
!pip install transformers accelerate

In [1]:
import os

In [2]:
%pwd

'/Users/vikkyfury/Desktop/pr/program/LangWiz-AI/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/vikkyfury/Desktop/pr/program/LangWiz-AI'

In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    data_path:Path
    model_ckpt:Path
    num_train_epochs = int
    evaluation_strategy= str
    save_strategy= str
    learning_rate= float
    per_device_train_batch_size= int
    per_device_eval_batch_size= int
    weight_decay= float
    save_total_limit= int
    predict_with_generate= bool
    fp16= bool
    push_to_hub= bool

In [ ]:
from LangWizAI.constants import *
from LangWizAI.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config =self.config.prepare_base_model
        params = self.params.TrainingArguements
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            evaluation_strategy = params.evaluation_strategy,
            save_strategy = params.save_strategy,
            learning_rate = params.learning_rate,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size= params.per_device_eval_batch_size,
            weight_decay= params.weight_decay,
            save_total_limit= params.save_total_limit,
            num_train_epochs= params.num_train_epochs,
            predict_with_generate= params.predict_with_generate,
            fp16 = params.fp16,
            push_to_hub = params.push_to_hub    
        )
        return prepare_base_model_config
        
        

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [ ]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_fr_en = AutoModelForSeq2SeqLM.from_pre_trained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model = model_fr_en)
        
        data_pt = load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs = self.config.num_train_epochs,
            evaluation_strategy = self.config.evaluation_strategy,
            save_strategy = self.config.save_strategy,
            learning_rate = self.config.learning_rate,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size= self.config.per_device_eval_batch_size,
            weight_decay= self.config.weight_decay,
            save_total_limit= self.config.save_total_limit,
            predict_with_generate= self.config.predict_with_generate,
            fp16= self.config.fp16,
            push_to_hub= self.config.push_to_hub
        )
        
        trainer = Trainer(
            model = model_fr_en, args = trainer_args,
            tokenizer = tokenizer, data_collator = seq2seq_data_collator,
            train_dataset = data_pt['train'],
            eval_dataset = data_pt['eval'] 
        )
        trainer.train()
        
        model_fr_en.save_pretrained(os.path.join(self.config.root_dir,'fr_en_model'))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'fr_en_tokenizer'))

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_prepare_base_model_config()
    model_trainer_config = PrepareBaseModel(config= model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e